<img src='otus.png'>

# Анализ текстовых данных

# Тематическое моделирование

Тема - это семантически однородный кластер текстов, характеризующийся специализированной терминологией.

Тематическая модель автоматически определяет, к каким темам относится каждый документ из коллекции документов, а также какие слова (термины) характеризуют каждую тему.

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d5/%D0%A2%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.png">

## Гипотезы тематического моделирования
- порядок слов в документе не важен
- порядок документов в коллекции не важен
- каждый терм w в документе связан с некоторой темой
- гипотеза условной независимости:
$$p(w|d,t) = p(w|t)$$

In [1]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 13.066s.


In [2]:
len(data_samples)

2000

In [3]:
data_samples[0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [4]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.799s.
Extracting tf features for LDA...
done in 1.060s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 1.496s.

Topics in NMF model (Frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope

In [5]:
lda.components_

array([[ 4.96604155,  4.3537397 , 21.42539886, ...,  1.57926488,
         1.33933502,  1.20988436],
       [ 0.48391034,  1.85845783, 14.04720958, ..., 74.59501615,
        59.36116266,  0.27698642],
       [ 0.18708486,  0.13728929,  0.31409364, ...,  1.02679042,
         2.56259123,  0.13662652],
       ...,
       [ 3.22343848, 39.1368944 , 11.24910558, ..., 23.37779481,
         3.06315114,  0.15230766],
       [ 1.41871388, 47.53082031, 16.14390001, ..., 82.46751192,
        16.51319941, 28.11660323],
       [ 4.02759659,  1.24781464, 13.26101699, ..., 29.0225105 ,
         0.24834416,  0.13033208]])

In [6]:
lda.components_.shape

(10, 1000)

# Наивный Байес
* Знаем метку каждого документа
* У каждого документа только одна метка  


Что можно сделать, если нет информации о метках?
#### Проблема кластеризации  
Можно решать с помощью EM-алгоритма:
* E-шаг - вычислить ожидания того, какой документ к какой теме относится
* M-шаг - с помощью Наивного Байеса определить вероятности $p(w|t)$ при фиксированных метках


## EM-алгоритм

Решает задачу кластеризации.  
Подбирает некоторые параметры модели для данных в которых неизвестен ответ.  

Expectation шаг:
* зафиксировать параметры модели
* посчитать значения скрытых переменных
Maximization шаг:
* зафиксировать скрытые переменные
* посчитать параметры модели

Повторять до сходимости.

Есть математическое обоснование того, что метод сходится к локальному экстремуму, на каждом шаге значение функции правдоподобия не убывает (правдоподобие $p(\theta | \mathcal{X})$ - насколько правдоподобна модель при данных параметрах, насколдько она хорошо описывает данные)

Частный случай EM-алгоритма - **k-means**.  
Метки кластеров - скрытые переменные Z  (latent variables)  
Параметры модели - центры кластеров  

<img src="kmeans.png">

Еще вариант EM-алгоритма - разделение смеси гауссиан (Gaussian Mixture Model, GMM)

Параметры модели - центр кластера и матрица ковариаций (здесь описывает форму могомерного нормального распраделения, или гауссианы)
Скрытые переменные - вероятность пренадлежности к каждой гауссиане (метка кластера выбирается как наиболее вероятный кластер)


<img src="gauss.png">

## PLSA

Что если у каждого документа может быть много меток?

Рассмотрим модель:
* Каждое слово в документе $d$ сгенерировано из некоторой темы $t \in T$
* Документ сгенерирован некоторым распределением над темами $p(t|d)$
* Слово сгенерировано из темы (не из документа) $p(w|d, t) = p(w|d)$
* Получаем правдоподобие: $$p(w|d) = \sum_{t \in T}p(w|t)p(t|d) $$

Полученная модель - probabilistic latent semantic analysis, pLSA, Вероятностный латентно-семантический анализ

http://www.machinelearning.ru/wiki/index.php?title=%D0%92%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%BD%D1%8B%D0%B9_%D0%BB%D0%B0%D1%82%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%B5%D0%BC%D0%B0%D0%BD%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7

Обучение:  
Нам нужны величины:
* $p(w|t)$ - вероятности слов в темах, обозначим $\phi_{wt}$

* $p(t|d)$ - вероятности тем в документах, обозначим $\theta_{td}$

E-шаг:
* фиксируем $\phi_{wt}$ и $\theta_{td}$
* вычисляем $$p(t|d,w) = \frac{\phi_{wt} \theta_{td}}{\sum_{s \in T}\phi_{ws} \theta_{sd}}$$ для всех тем, для каждого документа, для каждого термина
* вычисляем количество терминов, которое генерируется в документе $d$ темой $t$ $$n_{dwt} = n_{dw}p(t|d,w)$$

М-шаг:
* по вычисленным $p(t|d,w)$ обновить приближения модели $\phi_{wt}$ и $\theta_{td}$
* $$n_{wt} = \sum_d n_{dwt}$$ $$n_{td} = \sum_{w \in d} n_{dwt}$$ $$n_t=\sum_w n_{wt}$$
* $$\theta_{td} = \frac{n_{td}}{n_d}$$ $$\phi_{wt} = \frac{n_{wt}}{n_t}$$


Можно не хранить матрицу $n_{dwt}$, а итерироваться по документам и суммировать $n_{wt}$ и $n_{td}$
* Много локальных экстремумов
* Много параметров, модель переобучается
* Нужно достичь не локальный минимум, а добиться интерпретируемости - найти "хороший" минимум

## LDA

В общем случае, чтобы улучшить pLSA, в логарифм правдоподобия добавляют регуляризацию:

$$\sum_{d \in D} \sum_{w \in d} n_{dw} ln \sum_{t \in T} \phi_{wt} \theta_{td} + \sum_i \tau_i R_i(\Phi, \Theta)$$

Если добавить априорное распределение - распределение дирехле, получим алгоритм LDA - Latent Dirichlet Allocation

В итоге получаем "хорошее" интерпретируемое решение (лучше, чем с pLSA)


Один документ может содержать несколько тем.  
Составляем иерархическую модель:  
* первый уровень - смесь, компоненты которой отвечают за темы
* второй уровень - мультиномиальная переменная с априорным распределением Дирихле, которая определяет "распределение над темами" в документе

Обучение:
* сэмплирование по Гибсу
* online variational bayes

## Cхема решения задач NLP:
1. Сбор данных:
    - API
    - crawling сайта
    - известные датасеты
    - Mechanical Turk, Toloka, etc.

2. Выбор метрики качества.

3. Предобработка данных.

4. Векторизация данных.
    - Bag Of Words
    - TF-IDF
    - Word-Embeddings (Word2Vec, Glove, Fasttext)
    - Text embeddings (ELMO, Infersent)

5. Выбор модели.
    - GridSerch
    - RandomSearch

В первую очередь стоит смотреть на линейные модели и нейронные сети.

6. Оценка решения в продакшене.

7. GoTo 1.

### Дополнительные материалы

http://www.machinelearning.ru/wiki/images/8/82/BMMO11_14.pdf  
http://www.machinelearning.ru/wiki/images/f/f7/DirichletProcessNotes.pdf 

LDA material:
- http://www.cs.columbia.edu/~blei/talks/Blei_Topic_Modeling_Workshop_2013.pdf
- http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf
- http://www.machinelearning.ru/wiki/images/8/82/BMMO11_14.pdf
- http://www.machinelearning.ru/wiki/images/f/f7/DirichletProcessNotes.pdf 

coherence article:
- http://www.aclweb.org/anthology/N10-1012
